In [1]:
# Running %env without any arguments
# lists all environment variables

# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 0

import numpy as np
import pandas as pd
import io
import time
from datetime import datetime
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave, imshow, imresize
import tensorflow as tf
from tensorflow.python.platform import tf_logging
from tensorflow.contrib import layers
from tensorflow.contrib.training import add_gradients_summaries
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.training import optimizer as tf_optimizer
from tensorflow.python.ops import variables as tf_variables
import os.path
import tensorflow.contrib.slim as slim
import inception_preprocessing
import logging
import inception_v3_Last
import cdscount_preprocessing
import random

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'

#PRETRAINED_MODEL_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/inception-v3/20160828/inception_v3.ckpt'
PRETRAINED_MODEL_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/logs_focalloss/inception_v3_model.ckpt-882954'
LOG_PATH = DATASET_PATH + 'logs_last/'
LR_FILE_PATH = DATASET_PATH + 'logs_last/lr_setting/inception_lr_setting'
TRAIN_PATH = DATASET_PATH + 'Split/Train/'
#TRAIN_PATH = '/media/rs/FC6CDC6F6CDC25E4/resample_dataset2/'
VAL_PATH = DATASET_PATH + 'Split/Validation/'
TEST_PATH = DATASET_PATH + 'Test/'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
CATEGORY_WEIGHT_PATH = DATASET_PATH + 'catogory_with_weight.csv'

LEVEL1_NUM_LIST = [555, 441, 440, 237, 230, 230, 220, 206, 196, 184, 180, 162, 158, 137, 106, 104, 103, 101, 99, 89, 88, 85, 84, 83, 81, 74, 57, 50, 48, 45, 43, 42, 42, 38, 33, 33, 30, 29, 26, 25, 19, 16, 9, 6, 2, 1, 1, 1, 1]
# following is non-resample
LEVEL1_WEIGHT = [1.817, 1.528, 1.056, 2.174, 1.622, 2.531, 2.663, 1.103, 1.935, 1.937, 2.898, 1.279, 2.923, 1.095, 2.086, 3.0, 2.218, 3.0, 3.0, 2.123, 3.0, 1.038, 3.0, 3.0, 3.0, 3.0, 1.817, 1.448, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 1.955, 3.0, 1.946, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]

BATCH_SIZE = 160#128#256


IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
LEVEL1_CLASS = 49
LEVEL2_CLASS = 483
# validation examples num: 2319624
# train examples num: 10051704
# total step: 157057
TOTAL_EXAMPLES = 12371328

NUM_EPOCHES = 12
EPOCHES_OVER = 8

INPUT_THREADS = 12

initial_learning_rate = 0.01#0.0004
stop_learning_rate = 0.0000001
moving_average_decay = 0.96# use large to be more stable?
momentum = 0.9
num_steps_per_epoch = TOTAL_EXAMPLES / BATCH_SIZE + 1

In [3]:
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler(DATASET_PATH + 'tensorflow_inception_hierachy_train.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

In [4]:
def read_learning_rate(cur_step, num_steps_per_epoch):
    def inner_lr_parser(interval_start, interval_end, lr, dict_in, default_lr, use_epoch_percent, num_steps_per_epoch):
        lr = default_lr * lr
        if use_epoch_percent:
            interval_start = num_steps_per_epoch * interval_start
            interval_end = num_steps_per_epoch * interval_end
        interval_start = int(interval_start)
        interval_end = int(interval_end)
        if (interval_start < interval_end) and (lr > 0):
            dict_in[(interval_start, interval_end)] = lr
            
    lr_map = dict()
    default_lr = initial_learning_rate
    stop_lr = stop_learning_rate
    line_index = -1
    use_epoch_percent = True
    if os.path.exists(LR_FILE_PATH):
        with open(LR_FILE_PATH, 'r') as lr_setting_file:
            for _, line in enumerate(lr_setting_file):
                line = line.strip()
                if (line != '') and (not line.startswith('#')):
                    line_index += 1
                    if line_index == 0:
                        default_lr = float(line.split(':')[-1].strip())
                        continue
                    if line_index == 1:
                        stop_lr = float(line.split(':')[-1].strip())
                        continue
                    if line_index == 2:
                        use_epoch_percent = ('EPOCHES_PERCENT' in (line.split(':')[-1].strip()))
                        continue
                    # this is a list desciption
                    if line.startswith('['):
                        line = [float(s.strip()) for s in line[1:-1].strip().split()]
                        step_interval = (line[1] - line[0])/line[-1]
                        lr_interval = (line[3] - line[2])/line[-1]
                        begin = line[0]
                        lr_begin = line[2]
                        for index in range(int(line[-1])):
                            inner_lr_parser(begin, begin+step_interval, lr_begin, lr_map, default_lr, use_epoch_percent, num_steps_per_epoch)
                            begin += step_interval
                            lr_begin += lr_interval
                    else:
                        interval_start, interval_end, lr = [float(s) for s in line.strip().split()]
                        inner_lr_parser(interval_start, interval_end, lr, lr_map, default_lr, use_epoch_percent, num_steps_per_epoch)
    lr_ret = default_lr
#     print(use_epoch_percent)
    for (start, end), lr in lr_map.items():
        if (cur_step >= start) and (cur_step <= end):
            if (lr < lr_ret):
                lr_ret = lr
    if lr_ret < stop_lr: lr_ret = stop_lr      
    return lr_ret
# _ = read_learning_rate(1, num_steps_per_epoch)
# lr = []
# num_epoches_to_show = 10
# num_point = 100
# for i in [i*num_epoches_to_show*num_steps_per_epoch/num_point for i in range(num_point)]:
#     lr.append(read_learning_rate(i, num_steps_per_epoch))
# plt.plot(lr)
# plt.ylabel('learning rate')
# plt.show()

In [5]:
# def preprocess_for_inception(input_image, is_training = True):
#     return cdscount_preprocessing.preprocess_image(input_image, 180, 180, is_training)

In [6]:
class LabelMapping(object):
    def __init__(self, catogory_file_path):
        super(LabelMapping, self).__init__()
        self._category_level_csv = catogory_file_path
        self._category_map, self._category_level1_map, self._category_level2_map, self._len_level1, self._len_level2 = self.cvt_csv2tfrecord()
        
        assert (LEVEL1_CLASS == self._len_level1) and (LEVEL2_CLASS == self._len_level2), 'Other two levels are not mapped correctly.'
        self._catogory_weight_map = self.cvt_catogory_weight()
        self._mapping_strings = tf.constant( [ str(key) for key in self._category_map.keys() ] )
        #print(list(self._category_map.keys())[0])
        self._mapping_table = tf.contrib.lookup.index_table_from_tensor(mapping=self._mapping_strings, default_value=0) 
        
        self._level1_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level1_map.keys()), list(self._category_level1_map.values()), tf.int64, tf.int64), 0)
        self._level2_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level2_map.keys()), list(self._category_level2_map.values()), tf.int64, tf.int64), 0)
        self._weight_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._catogory_weight_map.keys()), list(self._catogory_weight_map.values()), tf.int64, tf.float32), 0)

    @property
    def category_map(self):
        return self._category_map
    @property
    def level1_table(self):
        return self._level1_table
    @property
    def level2_table(self):
        return self._level2_table
    @property
    def len_level1(self):
        return self._len_level1
    @property
    def len_level2(self):
        return self._len_level2
    @property
    def mapping_table(self):
        return self._mapping_table
    @property
    def weight_table(self):
        return self._weight_table
    
    def cvt_catogory_weight(self):
        category_weight_map = dict()
        csv = pd.read_csv(CATEGORY_WEIGHT_PATH).values
        for row in csv:  
            category_id, weight = row[0], row[2]
#             if weight > 1.5:
#                 weight = 1.5
            category_weight_map[int(category_id)] = 1.

        return category_weight_map

    def cvt_csv2tfrecord(self):
        level1_map, level2_map = self.create_level_map()
        count = 0
        category_map = dict()
        category_level1_map = dict()
        category_level2_map = dict()
        csv = pd.read_csv(self._category_level_csv).values
        for row in csv:  
            category_id, level1, level2 = row[0], row[1], row[2]
            category_map[category_id] = count
            category_level1_map[int(category_id)] = level1_map[level1]
            category_level2_map[int(category_id)] = level2_map[level2]
            count += 1

        return category_map, category_level1_map, category_level2_map, len(level1_map), len(level2_map)

    def create_level_map(self):
        csv = pd.read_csv(self._category_level_csv).values
        level_list = [list(), list()]
        for row in csv: 
            for level in range(1,3):
                if row[level] not in level_list[level-1]:
                    level_list[level-1].append(row[level])
        return dict(zip(level_list[0], range(len(level_list[0])))), dict(zip(level_list[1], range(len(level_list[1]))))

In [7]:
class CdiscountDataset(object):
    def __init__(self, data_path, file_begin_match, label_mapping, num_examples, num_classes, buffer_size, batch_size, is_training):
        super(CdiscountDataset, self).__init__()
        self._data_file_list = [ os.path.join(data_path, x) for x in os.listdir(data_path) if lambda x: os.path.isfile(x) and file_begin_match in x ]
        self._num_examples = num_examples
        self._num_classes = num_classes
        self._batch_size = batch_size
        self._buffer_size = buffer_size
        self._is_training = is_training
        self._category_map = label_mapping.category_map
        self._level1_table = label_mapping.level1_table
        self._level2_table = label_mapping.level2_table
        self._len_level1 = label_mapping.len_level1
        self._len_level2 = label_mapping.len_level2
        self._mapping_table = label_mapping.mapping_table
        self._weight_table = label_mapping.weight_table
        
    def __len__(self):
        return self._num_examples
    
#     def __getitem__(self, idx):
#         #print('read',idx)
#         try:
#             next_example, next_label, next_level0_label, next_level1_label = self._tf_sess.run(self.get_next())
#             #print(next_example, next_label, next_level0_label, next_level1_label)
#         except tf.errors.OutOfRangeError:
#             pass
#         return (torch.from_numpy(next_example), torch.from_numpy(next_label), torch.from_numpy(next_level0_label), torch.from_numpy(next_level1_label))
    
    @staticmethod
    def image_normalized(image):
        mean = [0.485, 0.456, 0.406 ]
        std  = [0.229, 0.224, 0.225 ]

        image = image.transpose((2,0,1))
        image = image.astype(float)/255.
        
        image[0] = (image[0] - mean[0]) / std[0]
        image[1] = (image[1] - mean[1]) / std[1]
        image[2] = (image[2] - mean[2]) / std[2]
        
        image = image.transpose((1,2,0))

        return image.astype(np.float32)

    @staticmethod
    def fix_center_crop(image, size=(160,160)):

        height, width = image.shape[0:2]
        w,h = size

        x0 = (width  -w)//2
        y0 = (height -h)//2
        x1 = x0 + w
        y1 = y0 + h
        image = image[y0:y1, x0:x1]

        return image
    @staticmethod
    def random_horizontal_flip(image, u=0.5):
        if random.random() < u:
            image = np.flip(image,1)  #np.fliplr(img) ##left-right
        return image
    @staticmethod
    def random_crop(image, size=(160,160), u=0.5):

        height,width=image.shape[0:2]
        w,h = size

        if random.random() < u:
            x0 = np.random.choice(width - w)
            y0 = np.random.choice(height - h)
        else:
            x0 = (width  -w)//2
            y0 = (height -h)//2

        x1 = x0 + w
        y1 = y0 + h
        image = image[y0:y1, x0:x1]

        return image
    @staticmethod
    def random_resize(image, scale_x_limits=[0.9, 1.1], scale_y_limits=[0.9, 1.1], u=0.5):
        if random.random() < u:
            height,width=image.shape[0:2]

            scale_x  = random.uniform(scale_x_limits[0],scale_x_limits[1])
            if scale_y_limits is not None:
                scale_y  = random.uniform(scale_y_limits[0],scale_y_limits[1])
            else:
                scale_y = scale_x

            w = int(scale_x*width )
            h = int(scale_y*height)

            image = imresize(image,(h,w))
        return image

    @staticmethod
    def _preprocess_by_pytorch_train(image):
        image = CdiscountDataset.random_resize(image, scale_x_limits=[0.9,1.1], scale_y_limits=[0.9,1.1], u=0.5)
        # flip  random ---------
        image = CdiscountDataset.random_crop(image, size=(160,160), u=0.5) 
        image = CdiscountDataset.random_horizontal_flip(image, u=0.5)
        return image.astype(np.float32)

    @staticmethod
    def _preprocess_by_pytorch_test(image):
        image  = CdiscountDataset.fix_center_crop(image, size=(160,160))  
        return image.astype(np.float32)
    @staticmethod
    def _preprocess_normalize(image):
        return CdiscountDataset.image_normalized(image)
    @staticmethod
    def _array_to_image_transform(image):
        mean = [0.485, 0.456, 0.406 ]
        std  = [0.229, 0.224, 0.225 ]
        if random.random() < 0.00001:
            image_to_save = image
            image_to_save[0] = image_to_save[0]*std[0] + mean[0]
            image_to_save[1] = image_to_save[1]*std[1] + mean[1]
            image_to_save[2] = image_to_save[2]*std[2] + mean[2]

            image_to_save = image_to_save*255
            image_to_save = np.transpose(image_to_save, (1, 2, 0))
            image_to_save = image_to_save.astype(np.uint8)
            imsave('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/Debug/{}.jpg'.format(int(random.random()*100000)), image_to_save)
        return image
    def _parse_function(self, example_proto):
        features = {'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
            'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
            'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))}
                
        parsed_features = tf.parse_single_example(example_proto, features)
        image = tf.image.decode_image(parsed_features["img_raw"])
        raw_label = parsed_features["category_id"]
        if self._is_training:
            image = tf.py_func(CdiscountDataset._preprocess_by_pytorch_train, [image], tf.float32, stateful=True)
            #tf.summary.image('final_train_image', tf.expand_dims(image, 0))
        else:
            image = tf.py_func(CdiscountDataset._preprocess_by_pytorch_test, [image], tf.float32, stateful=False)
            #tf.summary.image('final_test_image', tf.expand_dims(image, 0))
        image = tf.py_func(CdiscountDataset._preprocess_normalize, [image], tf.float32, stateful=False)
        #image = tf.py_func(CdiscountDataset._array_to_image_transform, [image], tf.float32, stateful=False)
        
        return image, tf.one_hot(self._mapping_table.lookup(tf.as_string(raw_label)), self._num_classes, axis=-1, dtype=tf.int64),\
                tf.one_hot(self._level1_table.lookup(raw_label), self._len_level1, axis=-1, dtype=tf.int64),\
                tf.one_hot(self._level2_table.lookup(raw_label), self._len_level2, axis=-1, dtype=tf.int64), self._weight_table.lookup(raw_label)
#         return image, self._mapping_table.lookup(tf.as_string(raw_label)),\
#                 self._level1_table.lookup(raw_label),\
#                 self._level2_table.lookup(raw_label)
#     @staticmethod
#     def _dddff(image):
#         image = image.transpose((2,0,1))
#         return image.astype(np.float32)
#     def _parse_function_22(self, example_proto):
#         features = {'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
#             'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
#             'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))}
                
#         parsed_features = tf.parse_single_example(example_proto, features)
#         image = tf.image.decode_image(parsed_features["img_raw"])
#         raw_label = parsed_features["category_id"]
#         image = cdscount_preprocessing.preprocess_image(image, 160, 160, self._is_training)
#         image = tf.py_func(CdiscountDataset._dddff, [image], tf.float32, stateful=False)
#         return image, tf.one_hot(self._mapping_table.lookup(tf.as_string(raw_label)), self._num_classes, axis=-1, dtype=tf.int64),\
#                 tf.one_hot(self._level1_table.lookup(raw_label), self._len_level1, axis=-1, dtype=tf.int64),\
#                 tf.one_hot(self._level2_table.lookup(raw_label), self._len_level2, axis=-1, dtype=tf.int64)

    def get_next(self):
        #next_example, next_label, next_level0_label, next_level1_label 
        return self._next_iter
    def create_dataset(self):
        self._dataset = tf.data.TFRecordDataset(self._data_file_list, compression_type='ZLIB', buffer_size = 409600)
        parse_func = lambda example : self._parse_function(example)
        self._dataset = self._dataset.map(parse_func, num_parallel_calls=INPUT_THREADS)
        self._dataset = self._dataset.prefetch(self._batch_size * 3)
        self._dataset = self._dataset.shuffle(buffer_size=self._buffer_size)
        self._dataset = self._dataset.batch(self._batch_size)
        # we don't want to repeat until finish training, instead we stop each one epoch finished
        #self._dataset = self._dataset.repeat(self._num_epochs)
        self._iterator = self._dataset.make_initializable_iterator()
        self._next_iter = self._iterator.get_next()
#             Compute for 100 epochs.
#             for _ in range(100):
#               sess.run(iterator.initializer)
#               while True:
#                 try:
#                   sess.run(next_element)
#                 except tf.errors.OutOfRangeError:
#                   break
#         map(
#             map_func,
#             num_threads=None,
#             output_buffer_size=None,
#             num_parallel_calls=None
#         )
#         Maps map_func across this datset. (deprecated arguments)

#         SOME ARGUMENTS ARE DEPRECATED. They will be removed in a future version. Instructions for updating: Replace num_threads=T with num_parallel_calls=T. Replace output_buffer_size=N with ds.prefetch(N) on the returned dataset.

#         Args:

#         map_func: A function mapping a nested structure of tensors (having shapes and types defined by self.output_shapes and self.output_types) to another nested structure of tensors.
#         num_threads: (Optional.) Deprecated, use num_parallel_calls instead.
#         output_buffer_size: (Optional.) A tf.int64 scalar tf.Tensor, representing the maximum number of processed elements that will be buffered.
#         num_parallel_calls: (Optional.) A tf.int32 scalar tf.Tensor, representing the number elements to process in parallel. If not specified, elements will be processed sequentially.
        return self._iterator.initializer


In [ ]:
def_graph = tf.Graph()
with def_graph.as_default() as graph:
    def train_step(input_examples, batch_labels, level1_onehot_labels, level2_onehot_labels, batch_weight):   
        with tf.name_scope("InceptionLast"):
            with slim.arg_scope(inception_v3_Last.inception_v3_arg_scope()):
                # here logits is the pre-softmax activations
                logits, end_points = inception_v3_Last.inception_v3(
                    input_examples,
                    num_classes = NUM_CLASS,
                    dropout_keep_prob=0.5,
                    is_training = True)
                
                #pre_logits = end_points['PreLogits']
                #pre_logits = tf.stop_gradient(end_points['PreLogits'])
                
                tvars = tf.trainable_variables()

                #org_vars = [var for var in tvars if 'level1_' not in var.name]
                org_head_only_vars = [var for var in tvars if (('Conv2d_1c_1x1' in var.name) or ('Mixed_7c' in var.name))]
                

                #variables_to_restore = slim.get_variables_to_restore()
                variables_to_restore = slim.get_variables_to_restore(exclude = ['InceptionV3/Logits', 'InceptionV3/AuxLogits'])
                                #                 variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay)
#                 variables_to_restore_ = variable_averages.variables_to_restore()

                #print(variables_to_restore)
    
                global_step = tf.train.get_or_create_global_step(graph = graph)
                
            
                loss_level3 = tf.losses.softmax_cross_entropy(onehot_labels = batch_labels, logits = logits, label_smoothing = 0.)
                #aux_loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = end_points['AuxLogits'], weights=0.2)
                total_loss = tf.losses.get_total_loss(add_regularization_losses=True)    # obtain the regularization losses as well

                custom_learning_rate = tf.placeholder(tf.float32, shape=[])
                #Now we can define the optimizer that takes on the learning rate
                #optimizer = tf.train.AdamOptimizer(learning_rate = custom_learning_rate)
                optimizer = tf.train.MomentumOptimizer(learning_rate = custom_learning_rate, momentum=momentum)

                #moving_average_variables = slim.get_model_variables()
                #variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
                # Use an alternative set of update ops in addition to the default updates:
                #tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, variable_averages.apply(moving_average_variables))


                #train_op_level3 = slim.learning.create_train_op(total_loss, optimizer, summarize_gradients=False, variables_to_train=org_vars)#, clip_gradient_norm=10)
                #train_op_level1 = slim.learning.create_train_op(total_loss, optimizer, summarize_gradients=False, variables_to_train=level1_only_vars)#, clip_gradient_norm=10)
                train_op = slim.learning.create_train_op(total_loss, optimizer, summarize_gradients=False, variables_to_train=org_head_only_vars)#, clip_gradient_norm=10)

                variables_to_restore_checkpoint = slim.get_variables_to_restore()

                #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.

                probabilities = end_points['Predictions']
                predictions = tf.argmax(probabilities, 1)
                target = batch_labels
                
                accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, tf.argmax(target, 1), name='train_accuracy')
                metrics_op = tf.group(accuracy_update)

                real_time_accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, tf.argmax(target, 1)), tf.float32))
                
                #Now finally create all the summaries you need to monitor and group them into one summary op.
                tf.summary.scalar('losses/Total_Loss', total_loss)
                tf.summary.scalar('accuracy', accuracy)
                tf.summary.scalar('train/real_time_accuracy', real_time_accuracy)
                tf.summary.scalar('learning_rate', custom_learning_rate)

                return train_op, global_step, metrics_op, variables_to_restore, variables_to_restore_checkpoint, predictions, custom_learning_rate, accuracy, real_time_accuracy, total_loss


In [ ]:
with def_graph.as_default() as graph:
    label_mapping = LabelMapping(CATEGORY_NAME_PATH)
    
    label_mapping = LabelMapping(CATEGORY_NAME_PATH)
    train_dataset = CdiscountDataset(TRAIN_PATH, 'output_file', label_mapping, TOTAL_EXAMPLES, NUM_CLASS, 10000, BATCH_SIZE, True)
    iterator_initalizer = train_dataset.create_dataset()

    #train_dataset = CdiscountDataset(TRAIN_PATH, '*output_file', label_mapping, TOTAL_EXAMPLES, NUM_CLASS, 20000, BATCH_SIZE, NUM_EPOCHES, True)

    batch_images, batch_labels, batch_labels_level1, batch_labels_level2, batch_weight = train_dataset.get_next()
   
    batch_images = tf.reshape(batch_images, [-1, 160, 160, 3])
    with tf.device('/gpu:0'):
        train_op, global_step, metrics_op, variables_to_restore, variables_to_restore_checkpoint, pred_op, lr, accuracy, real_time_accuracy, total_loss = train_step(batch_images, batch_labels, batch_labels_level1, batch_labels_level2, batch_weight)
         
    summary_op = tf.summary.merge_all()

    checkpoint_saver = tf.train.Saver(variables_to_restore_checkpoint)
    
    pre_train_saver = tf.train.Saver(variables_to_restore)

    def load_pretrain(sess):
        pre_train_saver.restore(sess, PRETRAINED_MODEL_PATH)

    # no need for specify local_variables_initializer and tables_initializer, Supervisor will do this via default local_init_op
    # init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer())
    init_op = tf.group(tf.global_variables_initializer())
    #init_op = tf.group(train_iterator_initializer, val_iterator_initializer, tf.global_variables_initializer())
    
    # Pass the init function to the supervisor.
    # - The init function is called _after_ the variables have been initialized by running the init_op.
    # - use default tf.Saver() for ordinary save and restore
    # - save checkpoint every 1.3 hours(4800)
    # - manage summary in current process by ourselves for memory saving
    # - no need to specify global_step, supervisor will find this automately
    # - initialize order: checkpoint -> local_init_op -> init_op -> init_func
    sv = tf.train.Supervisor(logdir=LOG_PATH, init_fn = load_pretrain, init_op = init_op, summary_op = None, saver = checkpoint_saver, save_model_secs=7200, checkpoint_basename='inception_model.ckpt')
    
    final_loss = 0.
    final_accuracy = 0.
    training_state = True

    cur_readed_lr = initial_learning_rate
    tf_logging.info(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
    #config.gpu_options.allow_growth = True
    with sv.managed_session(config=config) as sess:
    #with sv.prepare_or_wait_for_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
        sess.run(iterator_initalizer)
        # Here sess was either initialized from the pre-trained-checkpoint or
        # recovered from a checkpoint saved in a previous run of this code.
        for step in range(int(num_steps_per_epoch * NUM_EPOCHES)):         
            if sv.should_stop():
                tf_logging.info('Supervisor emit finished!')
                tf_logging.info('Current Loss: %s', loss)
                tf_logging.info('Current Accuracy: %s', accuracy)
                tf_logging.info('Saving current model to disk(maybe invalid).')
                training_state = False
                break

            start_time = time.time()

            if step % 1000 == 0:
                summ, cur_global_step = sess.run([summary_op, global_step], feed_dict={lr: cur_readed_lr})
                sv.summary_computed(sess, summ)
                if step > EPOCHES_OVER * num_steps_per_epoch:
                    raise StopIteration("over epoches reached.")
                cur_readed_lr = read_learning_rate(cur_global_step, num_steps_per_epoch)

            with tf.device('/gpu:0'):
                f, _, _, cur_loss, cur_acc, rt_accuracy, total_step, cur_lr = sess.run([batch_images,train_op, metrics_op, total_loss, accuracy, real_time_accuracy, global_step, lr], feed_dict={lr: cur_readed_lr})
            time_elapsed = time.time() - start_time

            if step % 10 == 0:
                #print(f)
                #break
                final_loss = cur_loss
                final_accuracy = cur_acc
                tf_logging.info('Current Speed: {:5.3f}sec/batch'.format(time_elapsed))
                tf_logging.info('Current Streaming Accuracy: {:5.3f}%'.format(cur_acc*100.))
                tf_logging.info('Current Realtime Accuracy: {:5.3f}%'.format(rt_accuracy*100.))
                tf_logging.info('Current Loss: {:5.3f}'.format(cur_loss))
                tf_logging.info('Epoch %s/%s, Global Step: %s', int(total_step / num_steps_per_epoch + 1), NUM_EPOCHES, total_step)
                tf_logging.info('Current Learning Rate: {}'.format(cur_lr))

        if training_state:
            #We log the final training loss and accuracy
            tf_logging.info('Final Loss: %s', final_loss)
            tf_logging.info('Final Accuracy: %s', final_accuracy)
            # Once all the training has been done, save the log files and checkpoint model
            tf_logging.info('Finished training! Model saved.')
        sv.saver.save(sess, sv.save_path, global_step = sv.global_step)
        tf_logging.info(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
INFO:tensorflow:2017-12-04 20:29:15
INFO:tensorflow:Restoring parameters from /media/rs/0E06CD1706CD0127/Kapok/kaggle/logs_focalloss/inception_v3_model.ckpt-882954
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path /media/rs/0E06CD1706CD0127/Kapok/kaggle/logs_last/inception_model.ckpt
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Current Speed: 9.481sec/batch
INFO:tensorflow:Current Streaming Accuracy: 0.000%
INFO:tensorflow:Current Realtime Accuracy: 0.000%
INFO:tensorflow:Current Loss: 12.792
INFO:tensorflow:Epoch 1/12, Global Step: 1
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.202sec/batch
INFO:tensorflow:Current Streaming Accuracy: 0.187%
INFO:tensorflow:Current Realtime Accuracy: 1.875%
INFO:tensorflow:Current Loss: 10.944
INFO:tensorflow:Epoch 1/12, Global Step: 11
INFO:tenso

INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.199sec/batch
INFO:tensorflow:Current Streaming Accuracy: 28.197%
INFO:tensorflow:Current Realtime Accuracy: 36.875%
INFO:tensorflow:Current Loss: 6.168
INFO:tensorflow:Epoch 1/12, Global Step: 261
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.197sec/batch
INFO:tensorflow:Current Streaming Accuracy: 28.345%
INFO:tensorflow:Current Realtime Accuracy: 36.250%
INFO:tensorflow:Current Loss: 6.665
INFO:tensorflow:Epoch 1/12, Global Step: 271
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.202sec/batch
INFO:tensorflow:Current Streaming Accuracy: 28.634%
INFO:tensorflow:Current Realtime Accuracy: 36.250%
INFO:tensorflow:Current Loss: 6.528
INFO:tensorflow:Epoch 1/12, Global Step: 281
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.196sec/batch
INFO:tensorflow:Current 

INFO:tensorflow:Current Streaming Accuracy: 33.012%
INFO:tensorflow:Current Realtime Accuracy: 48.750%
INFO:tensorflow:Current Loss: 5.748
INFO:tensorflow:Epoch 1/12, Global Step: 541
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.202sec/batch
INFO:tensorflow:Current Streaming Accuracy: 33.144%
INFO:tensorflow:Current Realtime Accuracy: 42.500%
INFO:tensorflow:Current Loss: 5.788
INFO:tensorflow:Epoch 1/12, Global Step: 551
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:global_step/sec: 4.62495
INFO:tensorflow:Current Speed: 0.202sec/batch
INFO:tensorflow:Current Streaming Accuracy: 33.244%
INFO:tensorflow:Current Realtime Accuracy: 39.375%
INFO:tensorflow:Current Loss: 6.067
INFO:tensorflow:Epoch 1/12, Global Step: 561
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.199sec/batch
INFO:tensorflow:Current Streaming Accuracy: 33.360%
INFO:tensorflow:Current Realtime Accu

INFO:tensorflow:Current Realtime Accuracy: 36.875%
INFO:tensorflow:Current Loss: 6.024
INFO:tensorflow:Epoch 1/12, Global Step: 821
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.198sec/batch
INFO:tensorflow:Current Streaming Accuracy: 35.466%
INFO:tensorflow:Current Realtime Accuracy: 44.375%
INFO:tensorflow:Current Loss: 5.515
INFO:tensorflow:Epoch 1/12, Global Step: 831
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.203sec/batch
INFO:tensorflow:Current Streaming Accuracy: 35.539%
INFO:tensorflow:Current Realtime Accuracy: 40.625%
INFO:tensorflow:Current Loss: 5.712
INFO:tensorflow:Epoch 1/12, Global Step: 841
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.208sec/batch
INFO:tensorflow:Current Streaming Accuracy: 35.589%
INFO:tensorflow:Current Realtime Accuracy: 37.500%
INFO:tensorflow:Current Loss: 6.186
INFO:tensorflow:Epoch 1/12, Global Step: 85

INFO:tensorflow:Epoch 1/12, Global Step: 1101
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.202sec/batch
INFO:tensorflow:Current Streaming Accuracy: 36.952%
INFO:tensorflow:Current Realtime Accuracy: 47.500%
INFO:tensorflow:Current Loss: 5.199
INFO:tensorflow:Epoch 1/12, Global Step: 1111
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.208sec/batch
INFO:tensorflow:Current Streaming Accuracy: 36.997%
INFO:tensorflow:Current Realtime Accuracy: 45.000%
INFO:tensorflow:Current Loss: 5.499
INFO:tensorflow:Epoch 1/12, Global Step: 1121
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Current Speed: 0.199sec/batch
INFO:tensorflow:Current Streaming Accuracy: 37.034%
INFO:tensorflow:Current Realtime Accuracy: 41.250%
INFO:tensorflow:Current Loss: 5.512
INFO:tensorflow:Epoch 1/12, Global Step: 1131
INFO:tensorflow:Current Learning Rate: 0.009999999776482582
INFO:tensorflow:Curre

...

INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.187sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.780%
INFO:tensorflow:Current Realtime Accuracy: 46.875%
INFO:tensorflow:Current Loss: 5.040
INFO:tensorflow:Epoch 1/12, Global Step: 10411
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.206sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.781%
INFO:tensorflow:Current Realtime Accuracy: 50.625%
INFO:tensorflow:Current Loss: 4.768
INFO:tensorflow:Epoch 1/12, Global Step: 10421
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.192sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.785%
INFO:tensorflow:Current Realtime Accuracy: 46.250%
INFO:tensorflow:Current Loss: 4.799
INFO:tensorflow:Epoch 1/12, Global Step: 10431
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.187sec/batch
INFO:tensorflow:Cu

INFO:tensorflow:Current Speed: 0.193sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.853%
INFO:tensorflow:Current Realtime Accuracy: 43.125%
INFO:tensorflow:Current Loss: 4.907
INFO:tensorflow:Epoch 1/12, Global Step: 10691
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.191sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.858%
INFO:tensorflow:Current Realtime Accuracy: 48.125%
INFO:tensorflow:Current Loss: 4.637
INFO:tensorflow:Epoch 1/12, Global Step: 10701
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.199sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.861%
INFO:tensorflow:Current Realtime Accuracy: 48.125%
INFO:tensorflow:Current Loss: 4.764
INFO:tensorflow:Epoch 1/12, Global Step: 10711
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.184sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.865%
INFO:tensorflow:Current Re

INFO:tensorflow:Current Streaming Accuracy: 45.949%
INFO:tensorflow:Current Realtime Accuracy: 46.250%
INFO:tensorflow:Current Loss: 4.760
INFO:tensorflow:Epoch 1/12, Global Step: 10971
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.194sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.953%
INFO:tensorflow:Current Realtime Accuracy: 50.625%
INFO:tensorflow:Current Loss: 4.535
INFO:tensorflow:Epoch 1/12, Global Step: 10981
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.187sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.955%
INFO:tensorflow:Current Realtime Accuracy: 51.875%
INFO:tensorflow:Current Loss: 4.354
INFO:tensorflow:Epoch 1/12, Global Step: 10991
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.367sec/batch
INFO:tensorflow:Current Streaming Accuracy: 45.959%
INFO:tensorflow:Current Realtime Accuracy: 51.875%
INFO:tensorflow:Curre

INFO:tensorflow:Current Realtime Accuracy: 46.875%
INFO:tensorflow:Current Loss: 4.640
INFO:tensorflow:Epoch 1/12, Global Step: 11251
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.186sec/batch
INFO:tensorflow:Current Streaming Accuracy: 46.030%
INFO:tensorflow:Current Realtime Accuracy: 45.000%
INFO:tensorflow:Current Loss: 4.618
INFO:tensorflow:Epoch 1/12, Global Step: 11261
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.194sec/batch
INFO:tensorflow:Current Streaming Accuracy: 46.033%
INFO:tensorflow:Current Realtime Accuracy: 50.000%
INFO:tensorflow:Current Loss: 4.436
INFO:tensorflow:Epoch 1/12, Global Step: 11271
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.186sec/batch
INFO:tensorflow:Current Streaming Accuracy: 46.037%
INFO:tensorflow:Current Realtime Accuracy: 44.375%
INFO:tensorflow:Current Loss: 4.958
INFO:tensorflow:Epoch 1/12, Global St

INFO:tensorflow:Current Loss: 4.314
INFO:tensorflow:Epoch 1/12, Global Step: 11531
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.199sec/batch
INFO:tensorflow:Current Streaming Accuracy: 46.109%
INFO:tensorflow:Current Realtime Accuracy: 48.750%
INFO:tensorflow:Current Loss: 4.424
INFO:tensorflow:Epoch 1/12, Global Step: 11541
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.201sec/batch
INFO:tensorflow:Current Streaming Accuracy: 46.112%
INFO:tensorflow:Current Realtime Accuracy: 45.625%
INFO:tensorflow:Current Loss: 4.725
INFO:tensorflow:Epoch 1/12, Global Step: 11551
INFO:tensorflow:Current Learning Rate: 0.009583333507180214
INFO:tensorflow:Current Speed: 0.191sec/batch
INFO:tensorflow:Current Streaming Accuracy: 46.115%
INFO:tensorflow:Current Realtime Accuracy: 46.875%
INFO:tensorflow:Current Loss: 4.631
INFO:tensorflow:Epoch 1/12, Global Step: 11561
INFO:tensorflow:Current Learning Rate: 0.